In [1]:
from pyspark.sql import SparkSession
import psycopg2
import pandas as pd
import os
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from pyspark import SparkConf

# initialise sparkContext

spark = SparkSession.builder.config("spark.jars", "postgresql-42.2.5.jar")\
    .config("spark.jars.packages","org.postgresql:postgresql:42.2.5")\
    .config('spark.driver.maxResultSize','20g').config("spark.executor.memory", "20g")\
    .config("spark.driver.memory","20g").config("spark.executor.memory","20g")\
    .config("spark.shuffle.file.buffer",'40').config("spark.scheduler.listenerbus.eventqueue.capacity","10000000")\
    .master("local").appName("tinkof-invest").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.sparkContext.setLogLevel("ERROR")
spark


22/05/20 16:41:25 WARN Utils: Your hostname, nikolay resolves to a loopback address: 127.0.1.1; using 192.168.1.3 instead (on interface wlp2s0)
22/05/20 16:41:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nikolay/anaconda3/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nikolay/.ivy2/cache
The jars for the packages stored in: /home/nikolay/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97c7dcf5-da34-4cca-96ea-8251517d7d68;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.5 in central
:: resolution report :: resolve 162ms :: artifacts dl 6ms
	:: modules in use:
	org.postgresql#postgresql;42.2.5 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-97c7dcf5-da34-4cca-96ea-8251517d7d68
	confs:

In [2]:
hour = spark.createDataFrame(pd.read_parquet('all_candles_hour.parquet'))

In [9]:
min_15 = spark.createDataFrame(pd.read_parquet('all_candles_15min.parquet'))

In [11]:
min_5 = spark.createDataFrame(pd.read_parquet('all_candles_5min.parquet'))

In [13]:
min_1 = spark.createDataFrame(pd.read_parquet('all_candles_1min.parquet'))

In [5]:
mode = "overwrite"
url = "jdbc:postgresql://localhost:5432/shares"
properties = {"user": "nikolay","password": "Andone1","driver": "org.postgresql.Driver"}


In [6]:
hour

DataFrame[figi: string, time: timestamp, open: decimal(14,8), close: decimal(14,8), high: decimal(14,8), low: decimal(14,8), volume: double, is_complete: double]

In [7]:
hour.write.jdbc(url=url, table="candles_hour", mode=mode, properties=properties)

In [10]:
min_15.write.jdbc(url=url, table="candles_15min", mode=mode, properties=properties)

In [12]:
min_5.write.jdbc(url=url, table="candles_5min", mode=mode, properties=properties)

In [14]:
min_1.write.jdbc(url=url, table="candles_1min", mode=mode, properties=properties)

In [9]:
pd.read_parquet('debug/BBG000Q7GG57_hour_from_2022-02-21_to2022-05-20.parquet')

,figi,time,open,close,high,low,volume,is_complete
0,None,2022-02-22 07:00:00+00:00,0.00364500,0.00340000,0.00364500,0.00300500,954,True
1,None,2022-02-22 08:00:00+00:00,0.00339500,0.00337000,0.00349000,0.00332500,293,True
2,None,2022-02-22 09:00:00+00:00,0.00341500,0.00343500,0.00344000,0.00336500,56,True
3,None,2022-02-22 10:00:00+00:00,0.00344000,0.00346500,0.00346500,0.00340000,26,True
4,None,2022-02-22 11:00:00+00:00,0.00344000,0.00345500,0.00349000,0.00344000,39,True
...,...,...,...,...,...,...,...,...
4,None,2022-05-19 11:00:00+00:00,0.00356000,0.00356000,0.00356000,0.00356000,2,True
5,None,2022-05-19 12:00:00+00:00,0.00356500,0.00356500,0.00356500,0.00356500,13,True
6,None,2022-05-19 13:00:00+00:00,0.00355000,0.00356500,0.00356500,0.00355000,10,True
7,None,2022-05-19 14:00:00+00:00,0.00355000,0.00353500,0.00356000,0.00353500,43,True
